In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH05-Model")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Model


# Caching

1. 개요 
동일한 입력에 대해 AI모델을 다시 호출하지 않고, 이전에 생성된 응답을 반환하는 기능
    

2. 목적
    - 일관된 응답 제공:
        - 동일한 입력에 대해 모델 출력이 변동될 수 있는 경우, 캐싱 통해 일관된 결과 반환
    - 응답 속도 향상:
        - 캐싱된 응답은 즉시 반환되므로, 모델 호출 없이도 빠른 응답 가능 
        - 실시간 응답이 중요한 어플리케이션(채팅봇)에서 활용
    - API 호출 최적화:
        - 동일한 입력에 대해 불필요한 API 호출을 줄여 서버 부하 감소 
        - API 호출 횟수 제한이 있는 환경에서 효과적
    - 비용 절감:
        - 모델 호출 시 발생하는 비용을 줄이기 위해 캐싱된 응답 재사용
        - API 호출 비용이 높은 엔터프라이즈 환경 또는 서비스에서 효과적

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [4]:
model = ChatOpenAI(
    temperature=1,
    model_name='gpt-3.5-turbo',
)

prompt = PromptTemplate.from_template(
    "{country}의 특산품에 대해서 200자 내외로 설명해 주세요."
)

chain = prompt | model

In [6]:
%%time

response = chain.invoke({'country':'루마니아'})
print(response.content)

루마니아는 다양한 특산품을 보유하고 있는 나라로, 그 중에서도 가장 유명한 것은 들머리우유와 카시쿠 레제와 같은 다양한 치즈 종류들이 있습니다. 특히 들머리우유는 신선하고 고소한 맛으로 유명하며, 요구르트, 버터, 그리고 다양한 유제품으로 가공되어 소비자들에게 제공됩니다. 또한 로즈 가득한 유칼리 워터와 달콤한 맛이 일품한 아칼라 처럼 다양한 와인과 주류 역시 루마니아의 특산품으로 유명합니다. 그 밖에도 호두, 베리, 채소, 그리고 수제 맥주와 같은 다양한 농산물과 음료가 나라 전역에서 생산되고 소비되고 있습니다.루마니아는 풍부한 자연환경과 다양한 농작물로 유명하며, 이러한 다양성은 이 나라의 특산품을 더욱 풍부하고 매력적으로 만들어 줍니다.
CPU times: user 19.8 ms, sys: 4.16 ms, total: 23.9 ms
Wall time: 3.62 s


# InMemoryCache

- 메모리를 이용해 입력과 출력의 매핑을 저장하는 방식  
- 가장 단순한 형태의 방식으로 데이터가 로컬 메모리에 저장되므로 응답 속도가 빠름 
- 주요 특징
    - 데이터 저장 위치:
        - 어플리케이션 실행 중 메모리에 저장
        - 어플리케이션 종료되면 데이터가 사라지는 휘발성
    - 제한된 용량:
        - 메모리 크기에 의존적 
        - 비교적 작은 규모의 어플리케이션에 효과적 

(참고) 휘발성이 없도록 처리가 필요한 경우, SQLite Cache 설정

In [7]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

In [9]:
%%time

# 캐쉬 설정 
set_llm_cache(InMemoryCache())

# 체인 실행
response = chain.invoke({'country':'튀르키에'})
print(response.content)

튀르키는 다양한 특산품으로 유명한 나라로, 그 중에서도 특히 지중해 지방의 특산품이 인기가 많습니다. 올리브 오일은 튀르키의 대표적인 특산품 중 하나로, 풍부한 영양가와 특유의 향이 인기를 끌고 있습니다. 또한, 꿀과 열대과일, 허브 등도 튀르키의 대표적인 특산품으로 유명하며, 특히 로즈워터와 로즈오토는 튀르키의 전통적인 향기를 즐길 수 있는 제품으로 손님들에게 사랑받고 있습니다. 튀르키의 다채로운 특산품을 맛보고 경험해 보는 것은 특별한 여행 경험을 만들어줄 것입니다.
CPU times: user 14 ms, sys: 2.82 ms, total: 16.8 ms
Wall time: 2.51 s


In [10]:
%%time

# 체인 다시 실행
response = chain.invoke({'country':'튀르키에'})
print(response.content)

튀르키는 다양한 특산품으로 유명한 나라로, 그 중에서도 특히 지중해 지방의 특산품이 인기가 많습니다. 올리브 오일은 튀르키의 대표적인 특산품 중 하나로, 풍부한 영양가와 특유의 향이 인기를 끌고 있습니다. 또한, 꿀과 열대과일, 허브 등도 튀르키의 대표적인 특산품으로 유명하며, 특히 로즈워터와 로즈오토는 튀르키의 전통적인 향기를 즐길 수 있는 제품으로 손님들에게 사랑받고 있습니다. 튀르키의 다채로운 특산품을 맛보고 경험해 보는 것은 특별한 여행 경험을 만들어줄 것입니다.
CPU times: user 1.76 ms, sys: 47 μs, total: 1.8 ms
Wall time: 1.8 ms


# SQLite Cache

- SQLite : 경량의 파일 기반 관계형 데이터베이스
- 주요 특징
    - 데이터 저장 위치:
        - 데이터는 SQLite DB에 저장되고, 애플리케이션 종료 후에도 보관됨 
        - 기본적으로 로컬 디스크에 저장 
    - 영구적 캐싱:
        - InMemoryCache와 달리 애플리케이션 종료 후에도 데이터 유지 

In [11]:
import os
from langchain_community.cache import SQLiteCache

In [12]:
# 캐시 디렉토리 설정 
if not os.path.exists("cache"):
    os.makedirs("cache")

# 캐시 설정 
set_llm_cache(
    SQLiteCache(database_path="cache/llm_cache.db")
)

In [13]:
%%time

# 체인 실행
response = chain.invoke({'country':'이집트'})
print(response.content)

이집트의 특산품으로는 향긋한 차, 화려한 애굽 유리공예품, 신비로운 향수 등이 유명합니다. 특히 이집트 차는 향긋하고 진한 맛으로 유명하며, 이탈리아와 프랑스에 수출되어 인기를 끌고 있습니다. 또한 이집트의 애굽 유리공예품은 다양한 디자인과 색상으로 세계적인 인기를 얻고 있으며, 관광객들에게 인기 있는 기념품으로 자리 잡고 있습니다. 그리고 이집트의 향수는 오랜 전통을 가지고 있어 고품질의 원료로 만들어지며, 독특하고 강렬한 향기로 유명합니다. 이들 제품들은 이집트의 역사와 문화를 담고 있어 많은 이들로부터 사랑을 받고 있습니다.
CPU times: user 18.3 ms, sys: 4.33 ms, total: 22.7 ms
Wall time: 2.69 s


In [14]:
%%time

# 체인 다시 실행
response = chain.invoke({'country':'이집트'})
print(response.content)

이집트의 특산품으로는 향긋한 차, 화려한 애굽 유리공예품, 신비로운 향수 등이 유명합니다. 특히 이집트 차는 향긋하고 진한 맛으로 유명하며, 이탈리아와 프랑스에 수출되어 인기를 끌고 있습니다. 또한 이집트의 애굽 유리공예품은 다양한 디자인과 색상으로 세계적인 인기를 얻고 있으며, 관광객들에게 인기 있는 기념품으로 자리 잡고 있습니다. 그리고 이집트의 향수는 오랜 전통을 가지고 있어 고품질의 원료로 만들어지며, 독특하고 강렬한 향기로 유명합니다. 이들 제품들은 이집트의 역사와 문화를 담고 있어 많은 이들로부터 사랑을 받고 있습니다.
CPU times: user 3.59 ms, sys: 330 μs, total: 3.92 ms
Wall time: 3.74 ms


-----
** End of Documents **